# 1. Import the necessary libraries for you to be able to query 4square API

In [51]:
import requests
import json
import pandas as pd
from getpass import getpass

import os
import requests
import json
from dotenv import load_dotenv
import pandas as pd

import geopandas as gpd
from cartoframes.viz import Map, Layer, popup_element

token = os.getenv("token")

# 2. Get names and location of the 10 coffee places by passing Ironhack as your location

In [52]:
where = "Pamplona 96, 08018 Barcelona"

import time
def gets_coordinates_from_string (where):
    url_geocode = f"https://geocode.xyz/{where}?json=1"
    res = requests.get(url_geocode)
    
    try:
        return res.json()
    
    except:
        time.sleep(2)
        gets_coordinates_from_string (where)

In [60]:
res = gets_coordinates_from_string(where)
res

{'standard': {'stnumber': '96',
  'addresst': 'Pamplona',
  'statename': 'Catalonia',
  'postal': '08018',
  'region': 'Catalonia',
  'prov': 'ES',
  'city': 'Barcelona',
  'countryname': 'Spain',
  'confidence': '0.90'},
 'longt': '2.19035',
 'alt': {},
 'elevation': {},
 'latt': '41.39780'}

In [63]:
lat, lon = res["latt"], res["longt"]
lat

'41.39780'

In [65]:
import requests
def find_close (query, lat, lon, radius = 1000, limit = 10):
    url = f"https://api.foursquare.com/v3/places/search?query={query}&ll={lat}%2C{lon}&radius={radius}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": "fsq3zJlP+zylHII9gRWfq9KurwhB8+0LPJw/XnnFyamdXaE="
    }

    response = requests.get(url, headers=headers)

    return response.json()

find_close("cafe", lat, lon)


{'results': [{'fsq_id': '5ba397333fcee8002ce9d8bc',
   'categories': [{'id': 13037,
     'name': 'Cafeteria',
     'short_name': 'Cafeteria',
     'plural_name': 'Cafeterias',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/education/cafeteria_',
      'suffix': '.png'}}],
   'chains': [],
   'closed_bucket': 'LikelyOpen',
   'distance': 440,
   'geocodes': {'drop_off': {'latitude': 41.399274, 'longitude': 2.195294},
    'main': {'latitude': 41.399175, 'longitude': 2.195368},
    'roof': {'latitude': 41.399175, 'longitude': 2.195368}},
   'link': '/v3/places/5ba397333fcee8002ce9d8bc',
   'location': {'address': 'Calle de Pallars, 154',
    'admin_region': 'Cataluña',
    'country': 'ES',
    'cross_street': '',
    'formatted_address': 'Calle de Pallars, 154, 08005 Barcelona Catalunya',
    'locality': 'Barcelona',
    'postcode': '08005',
    'region': 'Catalunya'},
   'name': 'Cafeteria Industrial',
   'related_places': {},
   'timezone': 'Europe/Madrid'},
  {'fsq_id'

# 3. Now get the 5 CLOSEST coffe places. Make use of params: limit, near?

In [72]:
def find_closest (query, near = where, limit = 5):
    url = f"https://api.foursquare.com/v3/places/search?query={query}&near={where}&limit={limit}"

    headers = {
        "accept": "application/json",
        "Authorization": "fsq3zJlP+zylHII9gRWfq9KurwhB8+0LPJw/XnnFyamdXaE="
    }

    response = requests.get(url, headers=headers)

    return response.json()

find_closest("cafe", where)

{'results': [{'fsq_id': '5ce13c6cfc9e94002c5a81c5',
   'categories': [{'id': 13034,
     'name': 'Café',
     'short_name': 'Café',
     'plural_name': 'Cafés',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/cafe_',
      'suffix': '.png'}},
    {'id': 13065,
     'name': 'Restaurant',
     'short_name': 'Restaurant',
     'plural_name': 'Restaurants',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/default_',
      'suffix': '.png'}}],
   'chains': [],
   'closed_bucket': 'VeryLikelyOpen',
   'distance': 855,
   'geocodes': {'drop_off': {'latitude': 41.396161, 'longitude': 2.156803},
    'main': {'latitude': 41.396295, 'longitude': 2.15692},
    'roof': {'latitude': 41.396295, 'longitude': 2.15692}},
   'link': '/v3/places/5ce13c6cfc9e94002c5a81c5',
   'location': {'address': 'Carrer de Minerva, 2',
    'admin_region': 'Cataluña',
    'country': 'ES',
    'cross_street': '',
    'formatted_address': 'Carrer de Minerva, 2, 08006 Barcelona Cata

This didn't work, they are further than the ones in question 2. I am going to bed

# 4. Define a function that returns name, lat, long for starbucks OR restaurants AND for any given location

In [57]:
def foursquare_places (venue, coordinates):
    pass

# 5. Export it into data folder as a json: `data/coffe_shops.json`

In [58]:
# your code here

# BONUS: load it into MongoDB in a collection named "ironcoffee" using pymongo

In [59]:
# your code here